<a href="https://colab.research.google.com/github/DrJamali/Ai-Project/blob/main/top10_recommendation_linkedin_job_posting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install sentence-transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab import files
files.upload()  # This will prompt you to upload `kaggle.json`
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


Saving kaggle.json to kaggle.json


In [ ]:
!pip install kaggle
!mkdir -p /content/datasets

# Download LinkedIn job postings dataset
!kaggle datasets download -d arshkon/linkedin-job-postings -p /content/datasets
!unzip -q /content/datasets/linkedin-job-postings.zip -d /content/datasets/linkedin_job_postings

print('Data source import complete.')

Dataset URL: https://www.kaggle.com/datasets/arshkon/linkedin-job-postings
License(s): CC-BY-SA-4.0
 89% 142M/159M [00:00<00:00, 210MB/s]
100% 159M/159M [00:00<00:00, 181MB/s]
Data source import complete.


In [ ]:
!pip install nltk
import nltk
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sentence_transformers import SentenceTransformer, util
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics.pairwise import cosine_similarity

# Download necessary NLTK data files


# import warnings
# warnings.filterwarnings('ignore')

NameError: name 'nltk' is not defined

In [ ]:
df = pd.read_csv('/content/datasets/linkedin_job_postings/postings.csv')

In [ ]:
df.head()

,job_id,company_name,title,description,max_salary,pay_period,location,company_id,views,med_salary,...,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,normalized_salary,zip_code,fips
0,921716,Corcoran Sawyer Smith,Marketing Coordinator,Job descriptionA leading real estate firm in N...,20.0,HOURLY,"Princeton, NJ",2774458.0,20.0,NaN,...,Requirements: \n\nWe are seeking a College or ...,1.713398e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,38480.0,8540.0,34021.0
1,1829192,NaN,Mental Health Therapist/Counselor,"At Aspen Therapy and Wellness , we are committ...",50.0,HOURLY,"Fort Collins, CO",NaN,1.0,NaN,...,NaN,1.712858e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,83200.0,80521.0,8069.0
2,10998357,The National Exemplar,Assitant Restaurant Manager,The National Exemplar is accepting application...,65000.0,YEARLY,"Cincinnati, OH",64896719.0,8.0,NaN,...,We are currently accepting resumes for FOH - A...,1.713278e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,55000.0,45202.0,39061.0
3,23221523,"Abrams Fensterman, LLP",Senior Elder Law / Trusts and Estates Associat...,Senior Associate Attorney - Elder Law / Trusts...,175000.0,YEARLY,"New Hyde Park, NY",766262.0,16.0,NaN,...,This position requires a baseline understandin...,1.712896e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,157500.0,11040.0,36059.0
4,35982263,NaN,Service Technician,Looking for HVAC service tech with experience ...,80000.0,YEARLY,"Burlington, IA",NaN,3.0,NaN,...,NaN,1.713452e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,70000.0,52601.0,19057.0


In [ ]:
df.shape

(123849, 31)

In [ ]:
df.isnull().sum()

,0
job_id,0
company_name,1719
title,0
description,7
max_salary,94056
pay_period,87776
location,0
company_id,1717
views,1689
med_salary,117569


In [ ]:
# Select relevant columns and drop duplicates
df = df[['title', 'description']].drop_duplicates().dropna()

# Preprocessing

In [ ]:
# Preprocess text data
nltk.download('punkt_tab')
def preprocess_text(text):
    # lowecasing
    text = text.lower()
    # remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # tokenize
    words = word_tokenize(text)
    # remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    # lemmatization the words
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]

    # Join the words back into a full string
    preprocess_text = ' '.join(words)

    return preprocess_text

df['description'] = df['description'].apply(preprocess_text)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


NameError: name 'df' is not defined

# Split Into Train and Test Sets

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

### Random Sampling

In [ ]:
sample_train_df = train_df.sample(frac=0.005, random_state=42)

In [ ]:
sample_train_df.shape

NameError: name 'sample_train_df' is not defined

In [ ]:
# Define the pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english')),
    ('clf', LogisticRegression(solver='liblinear', class_weight='balanced'))
])

# Parameter grid
param_grid = {
    'tfidf__max_df': [0.75, 0.85, 1.0],
    'tfidf__min_df': [1, 2, 5],
    'tfidf__ngram_range': [(1, 1), (1, 2)],
    'clf__C': [0.1, 1, 10]
}

# Perform RandomizedSearchCV
grid_search = RandomizedSearchCV(pipeline, param_grid, cv=2, verbose=2, n_jobs=-1, n_iter=10)
grid_search.fit(sample_train_df['description'], sample_train_df['title'])

# Update vectorizer with best parameters
best_params = grid_search.best_params_
vectorizer = TfidfVectorizer(stop_words='english',
                             max_df=best_params['tfidf__max_df'],
                             min_df=best_params['tfidf__min_df'],
                             ngram_range=best_params['tfidf__ngram_range'])

# Fit the vectorizer on the full dataset
tfidf_matrix = vectorizer.fit_transform(sample_train_df['description'])

Fitting 2 folds for each of 10 candidates, totalling 20 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(


In [ ]:
from sentence_transformers import SentenceTransformer, util

# Reset the index of the DataFrame
sample_train_df = sample_train_df.reset_index(drop=True)

# load pre-trained model
model = SentenceTransformer('all-mpnet-base-v2')

# encode job description
job_embeddings = model.encode(sample_train_df['description'].tolist(), convert_to_tensor=True)

model.safetensors:  91%|######### | 398M/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def search_jobs(query, job_embeddings, df, model, vectorizer, top_n=10):
    # Preprocess and encode the query with BERT
    query = preprocess_text(query)
    query_embedding = model.encode(query, convert_to_tensor=True)

    # Compute cosine similarity with BERT embeddings
    cosine_scores_bert = util.pytorch_cos_sim(query_embedding, job_embeddings)[0]

    # Transform query with TF-IDF vectorizer and compute cosine similarity
    query_tfidf = vectorizer.transform([query])
    cosine_scores_tfidf = cosine_similarity(query_tfidf, tfidf_matrix).flatten()

    # Combine scores (e.g., weighted sum)
    combined_scores = 0.5 * cosine_scores_bert.cpu().numpy() + 0.5 * cosine_scores_tfidf

    # Get top N most similar jobs
    top_indices = np.argsort(combined_scores)[-top_n:][::-1]
    top_jobs = df.iloc[top_indices]

    return top_jobs

In [ ]:
# Make predictions on the test set
predicted_labels = grid_search.predict(test_df['description'])
true_labels = test_df['title']

# Define evaluation function
def evaluate_model(true_labels, predicted_labels):
    accuracy = accuracy_score(true_labels, predicted_labels)
    precision = precision_score(true_labels, predicted_labels, average='weighted')
    recall = recall_score(true_labels, predicted_labels, average='weighted')
    f1 = f1_score(true_labels, predicted_labels, average='weighted')
    print(f"Accuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1-Score: {f1}")

# Evaluate the model
evaluate_model(true_labels, predicted_labels)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.013300270513976555
Precision: 0.00423318109300581
Recall: 0.013300270513976555
F1-Score: 0.005447561700419916


In [ ]:
query = 'i love software engineer, i have bachelor degree on computer science'
top_jobs = search_jobs(query, job_embeddings, df, model, vectorizer)
top_jobs

,title,description
298,Staff Accountant,maval industry llc environmental automotive ma...
350,Social Media Marketing Intern,college student passionate social medium marke...
100,Construction Project Manager,primary duty include developing project budget...
443,Pool Design and Sales Representative,sale design professional job type fulltime com...
153,Group Fitness Instructor,company descriptionburn boot camp fitness faci...
171,User Experience Designer,company descriptionstarup insurance personaliz...
33,Salesperson,abme leader corporate event planning looking m...
400,Accounting Manager,accountingoffice managerwhy choose smssandston...
249,Director Origination,brandlab holding bfv llp private equity firm i...
269,2 Way Radio Installer,company descriptionarcticom llc bsnc company l...


In [ ]:
import joblib

# Save the trained model and vectorizer
joblib.dump(grid_search, 'job_search_model.pkl')
joblib.dump(vectorizer, 'vectorizer.pkl')
joblib.dump(model, 'sentence_transformer_model.pkl')

['sentence_transformer_model.pkl']